In [1]:
# Okay, in this work, we will still not be storing the extracted features that we will be using in the stage 2 model, but rather focus on the model performnace, incase we need to report this somewhere.

In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

2025-08-28 10:26:30.386517: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-28 10:26:30.423562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-28 10:26:30.423586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-28 10:26:30.424481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 10:26:30.430311: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# import the model here?

In [ ]:
model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping.keras")

In [ ]:
model.summary()

In [ ]:
# we might even need the data generator for the predictions - hold on to this thought for now